In [ ]:
## Predicting labels of unlabeled data

In [1]:
import pickle
#loaded_model = pickle.load(open('../pickled_models/19.5.2_4u.RF.pkl', 'rb'))
loaded_model = pickle.load(open('../pickled_models/19.5.2_4u.RF.pkl', 'rb'))
# 0-unk, 1-patho, 2-ben
print("loaded model")

loaded model


In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

df = pd.read_pickle("../pickled/chr1-4_nolabels.pkl")
df= df.drop(columns=['clinvar_clnsig'])
#answers = df.pop('Problematic') 

print("shape before", df.shape)
df = df.replace({'.': 0})  
df = df.dropna()
print("shape after dropping all data with score = '.'", df.shape)


df = df.replace({'-': 0}) 
print("shape after dropping Unk", df.shape)
df.head()

shape before (3988, 8)
shape after dropping all data with score = '.' (3988, 8)
shape after dropping Unk (3988, 8)


aapos codonpos  Polyphen2_HVAR_pred     GenoCanyon_score  \
chr pos    ref alt                                                             
1   65565  A   G        1        1                    0  2.01693589783425E-5   
    69453  G   C      121        3                    0  1.61590270936468E-5   
    69887  T   G      266        2                    0  4.18119972739408E-6   
    451053 C   G      209        2                    0                    0   
    451486 T   G       65        1                    0                    0   

                    LINSIGHT RVIS_ExAC ExAC_cnv.score  \
chr pos    ref alt                                      
1   65565  A   G    0.206807         0              0   
    69453  G   C           0         0              0   
    69887  T   G           0         0              0   
    451053 C   G           0         0              0   
    451486 T   G           0         0              0   

                    SORVA_LOF_MAF0.005_HetOrHom  
chr pos    ref alt                               
1   65565  A   G                       0.000399  
    69453  G   C                       0.000399  
    69887  T   G                       0.000399  
    451053 C   G                       0.000000  
    451486 T   G                       0.000000

In [18]:
#labels = df.pop('Problematic')

#forest = RandomForestClassifier(random_state=1, class_weight="balanced")
result = loaded_model.predict(df)
#inverse_transform(result)

df_result = df.copy()
df_result['prediction'] = result
#df_result['answers'] = answers
df_result.head(30)
#print(dir(result))


#print("PROBLEMATIC LABEL DISTRIBUTION \n", df.groupby('Problematic').count())
print("TOTAL SUM OF LABELS: \n", df_result['prediction'].value_counts())
print("PERCENT OF LABELS: \n", df_result['prediction'].value_counts(normalize=True))

TOTAL SUM OF LABELS: 
 0    2916
1     855
2     217
Name: prediction, dtype: int64
PERCENT OF LABELS: 
 0    0.731194
1    0.214393
2    0.054413
Name: prediction, dtype: float64


In [17]:
# 0-unk, 1-patho, 2-ben

df_patho = df_result[df_result['prediction'] ==1]
print(df_patho)

                      aapos codonpos  Polyphen2_HVAR_pred  \
chr pos      ref alt                                        
1   65565    A   G        1        1                    0   
    948130   C   G       -1        0                    0   
    962756   A   T      294        2                    0   
    963471   C   T      441        2                    0   
    965001   A   C      580        2                   -2   
    970952   G   T      186        3                    0   
    979530   C   G      500        3                    0   
    1041623  C   G      228        3                    0   
    1045359  G   T      653        2                    0   
    1050449  G   C     1529        1                    0   
    1051508  A   T     1809        2                    0   
    1071928  C   A      213        3                    0   
    1072383  A   G       62        1                    0   
    1282320  G   T       -1        0                    0   
    1311057  T   G      

# Look at arbitrary mutations:

### 1 924631	T	C
* high conservation
* no protein, gene, functinal anntoation
* not clear

### 4 40103016 A   T
* protein N4BP2 
* AGA (R) --> TGA (STOP) 
* premature truncation of protein, 
* probably bad

### 4 41494607 G   C
* slice site of a protein, intron donor GT --> CT wont work
* probably bad